# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=f216a456d0617373cb19b9df195c1c7a39b1adb8bade33fb3bd438afe65cfd0e
  Stored in directory: c:\users\vanif\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vanif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [1]:
from nltk import edit_distance

In [2]:
words = []
with open('litw-win.txt') as file:
    while True:
        line = file.readline()
        if not line:
            break
        words.append(line[8:].strip())
len(words)

162166

In [3]:
distance = [edit_distance('велечайшим', word, substitution_cost=2) for word in words]
words[distance.index(min(distance))]

'величайшим'

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [4]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2
import re

In [5]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''
snb_stemmer_ru = SnowballStemmer('russian')

w = re.compile('^[а-яА-ЯёЁ]*$')
[snb_stemmer_ru.stem(word) for word in word_tokenize(text) if w.search(word)]

['разб',
 'текст',
 'из',
 'формулировк',
 'втор',
 'задан',
 'на',
 'слов',
 'провед',
 'стемминг',
 'и',
 'лемматизац',
 'слов']

In [6]:
morph = pymorphy2.MorphAnalyzer()
pt = [morph.parse(word) for word in word_tokenize(text) if w.search(word)] 
[word[0].normalized.word for word in pt]

['разбить',
 'текст',
 'из',
 'формулировка',
 'второй',
 'задание',
 'на',
 'слово',
 'провести',
 'стемминг',
 'и',
 'лемматизация',
 'слово']

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize

In [8]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''
sentences = sent_tokenize(text)
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(sentences)
vectorizer.get_feature_names_out()

array(['второго', 'задания', 'из', 'лемматизацию', 'на', 'проведите',
       'разбейте', 'слов', 'слова', 'стемминг', 'текст', 'формулировки'],
      dtype=object)

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [9]:
import pandas as pd
import numpy as np

In [10]:
recipes = pd.read_csv('ru_recipes_sample.csv')
w = re.compile('[а-яА-ЯёЁ0-9 ]+')
recipes.description[0].lower()

'этот коктейль готовлю из замороженной клубники. если клубника свежая, то добавляю перепелиное яйцо, благодаря этому коктейль получается устойчиво густым.'

In [11]:
new_descr = recipes.description.apply(lambda text: " ".join([part.strip() for part in w.findall(text.lower())]))
recipes.description = new_descr
recipes.head()

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная кл...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...


### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [12]:
words = list(set([word for desc in recipes.description.apply(lambda descr: word_tokenize(descr)) for word in desc]))

In [13]:
import random

In [14]:
for _ in range(5):
    word1 = random.choice(words)
    word2 = random.choice(words)
    d = edit_distance(word1, word2, substitution_cost=2)
    print(f"d({word1}, {word2}) = {d}")

d(задавая, пенное) = 13
d(очередной, нахожусь) = 13
d(водного, обжаренного) = 8
d(выпекать, разместить) = 12
d(употребления, расцветку) = 17


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [15]:
def is_plagiarism(s1: str, s2: str) -> bool:
    words1 = word_tokenize(s1)
    words2 = word_tokenize(s2)
    P = 0
    for w1 in words1:
        for w2 in words2:
            d = edit_distance(w1, w2, substitution_cost=2)
            if(d<2):
                P+=1
    L = max(len(words1), len(words2))
    return P/L>0.8

In [16]:
rec_id = recipes.url.apply(lambda url: int(url.split('/')[-2]))
descr_135488 = recipes[rec_id == 135488].description.iloc[0]
descr_851934 = recipes[rec_id == 851934].description.iloc[0]
descr_135488, descr_851934

('прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами',
 'замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами')

In [17]:
is_plagiarism(descr_135488, descr_851934)

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [18]:
words = list(set([word for description in recipes.description.apply(lambda desc: word_tokenize(desc)) for word in description]))

df4 = pd.DataFrame(columns=['word', 'stemmed_word','normalized_word'])
df4.word = words

snb_stemmer_ru = SnowballStemmer('russian')
df4.stemmed_word = [snb_stemmer_ru.stem(word) for word in df4.word]

morph = pymorphy2.MorphAnalyzer()
df4.normalized_word = [morph.parse(word)[0].normalized.word for word in df4.word] 

df4 = df4.set_index('word')
df4.head()

,stemmed_word,normalized_word
word,,
растопленное,растоплен,растопить
омлетик,омлетик,омлетик
налимью,налим,налимий
насыщенном,насыщен,насыщенный
хрустящую,хрустя,хрустеть


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [19]:
from nltk.corpus import stopwords

In [20]:
ru_stopwords = stopwords.words('russian')

description_no_stopwords = recipes.description.apply(
    lambda desc: " ".join([
        word for word in word_tokenize(desc) if word not in ru_stopwords
    ])
)

recipes['description_no_stopwords'] = description_no_stopwords
recipes.head()

,url,name,ingredients,description,description_no_stopwords
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно,быстро вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная кл...,картофельное пюре куриные котлеты вкусная клас...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...


In [21]:
len(words)

16165

In [22]:
no_stopwords = [word for word in words if word not in ru_stopwords]
len(no_stopwords)

16014

In [23]:
#доля стопслов в уникальном наборе слов 
frac = (1-len(no_stopwords)/len(words))*100
f"{frac:.2f}%"

'0.93%'

In [24]:
all_words = [word for desc in recipes.description.apply(lambda descr: word_tokenize(descr)) for word in desc]
all_no_stopwords = [
    word for desc in recipes.description_no_stopwords.apply(lambda descr: word_tokenize(descr)) for word in desc
]

In [25]:
#доля стопслов в полном наборе слов
frac = (1-len(all_no_stopwords)/len(all_words))*100
f"{frac:.2f}%"

'32.54%'

In [26]:
from nltk.probability import FreqDist
fdist_before = FreqDist(all_words)
fdist_after = FreqDist(all_no_stopwords)

In [27]:
sorted(fdist_before.items(), key= lambda items: items[1], reverse=True)[:10]

[('и', 5062),
 ('в', 2592),
 ('с', 1942),
 ('на', 1658),
 ('очень', 1626),
 ('не', 1518),
 ('из', 1023),
 ('я', 982),
 ('а', 876),
 ('рецепт', 874)]

In [28]:
sorted(fdist_after.items(), key= lambda items: items[1], reverse=True)[:10]

[('очень', 1626),
 ('рецепт', 874),
 ('это', 743),
 ('блюдо', 527),
 ('вкусный', 461),
 ('просто', 439),
 ('вкусно', 380),
 ('приготовить', 344),
 ('вкус', 325),
 ('салат', 318)]

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [29]:
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)

In [30]:
pancakes = recipes[recipes.name.apply(lambda name: "оладьи" in name.lower())]
pancakes5 = pancakes.sample(5)
vectorizer = TfidfVectorizer()
vc = vectorizer.fit_transform(pancakes5.description)
df6 = pd.DataFrame(vc.toarray(), columns=vectorizer.get_feature_names_out())
df6

,ароматное,базилика,бежевого,белки,белого,блинчики,блюда,блюдо,больше,бы,...,чечевица,чтобы,шустрый,щеки,щепотка,этих,это,этом,яблочно,яркое
0,0.28805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.28805,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.28805
1,0.00000,0.147912,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.119335,...,0.000000,0.000000,0.000000,0.000000,0.147912,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.00000,0.000000,0.069895,0.000000,0.069895,0.069895,0.000000,0.00000,0.069895,0.056391,...,0.000000,0.069895,0.069895,0.069895,0.000000,0.069895,0.069895,0.056391,0.069895,0.00000
4,0.00000,0.000000,0.000000,0.172252,0.000000,0.000000,0.172252,0.00000,0.000000,0.000000,...,0.172252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.138972,0.000000,0.00000


In [31]:
df6.iloc[:1]

,ароматное,базилика,бежевого,белки,белого,блинчики,блюда,блюдо,больше,бы,...,чечевица,чтобы,шустрый,щеки,щепотка,этих,это,этом,яблочно,яркое
0,0.28805,0.0,0.0,0.0,0.0,0.0,0.0,0.28805,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28805


In [32]:
pancakes5.description.iloc[1]

'какой то хозяйственник проснулся в моём муже когда он принёс домой несколько килограммов помидоров теперь у нас дома начинаются томатные дни оладьи готовила полностью по рецепту от себя только щепотка базилика который как мне кажется помидорам всегда на пользу но я бы рецепт немного изменила выкладываю оригинальную версию со своими комментариями надеюсь вам понравится'

In [33]:
df6.iloc[1][[word for word in word_tokenize(pancakes5.description.iloc[1]) if word not in ru_stopwords]]

хозяйственник    0.147912
проснулся        0.147912
моём             0.147912
муже             0.147912
принёс           0.147912
домой            0.147912
несколько        0.147912
килограммов      0.147912
помидоров        0.147912
дома             0.147912
начинаются       0.147912
томатные         0.147912
дни              0.147912
оладьи           0.099058
готовила         0.147912
полностью        0.147912
рецепту          0.147912
щепотка          0.147912
базилика         0.147912
который          0.119335
кажется          0.147912
помидорам        0.147912
пользу           0.147912
рецепт           0.147912
немного          0.147912
изменила         0.147912
выкладываю       0.147912
оригинальную     0.147912
версию           0.147912
своими           0.147912
комментариями    0.147912
надеюсь          0.147912
понравится       0.147912
Name: 1, dtype: float64

7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [34]:
from numpy.linalg import norm

In [35]:
morph = pymorphy2.MorphAnalyzer()
w_regex = re.compile('^[а-яё]*$')

def vectorizer_n(string):
    return [
        morph.parse(word)[0].normalized.word
        for word in word_tokenize(string.lower())
        if w_regex.search(word)
    ]


In [36]:
cv = CountVectorizer(tokenizer=vectorizer_n, stop_words=ru_stopwords)

recipes_cv = cv.fit_transform(pancakes5.description)
recipes_arr = recipes_cv.toarray()

recipes_arrn = recipes_arr / norm(recipes_arr, axis=1)[:, np.newaxis]

temp = recipes_arrn @ recipes_arrn.T
temp

D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


array([[1.        , 0.        , 0.        , 0.        , 0.11547005],
       [0.        , 1.        , 0.04622502, 0.02444119, 0.03042903],
       [0.        , 0.04622502, 1.        , 0.20336296, 0.05063697],
       [0.        , 0.02444119, 0.20336296, 1.        , 0.06693494],
       [0.11547005, 0.03042903, 0.05063697, 0.06693494, 1.        ]])

In [37]:
from sklearn.metrics.pairwise import cosine_distances as cos_d

In [38]:
1 - cos_d(recipes_arr)

array([[1.        , 0.        , 0.        , 0.        , 0.11547005],
       [0.        , 1.        , 0.04622502, 0.02444119, 0.03042903],
       [0.        , 0.04622502, 1.        , 0.20336296, 0.05063697],
       [0.        , 0.02444119, 0.20336296, 1.        , 0.06693494],
       [0.11547005, 0.03042903, 0.05063697, 0.06693494, 1.        ]])

Попробуем не через описание а через ингридиенты

In [39]:
cv = CountVectorizer(tokenizer=vectorizer_n, stop_words=ru_stopwords)

recipes_cv = cv.fit_transform(pancakes5.ingredients)
1 - cos_d(recipes_cv.toarray())

D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


array([[1.        , 0.82172602, 0.72045197, 0.75167814, 0.29534288],
       [0.82172602, 1.        , 0.62103443, 0.59160798, 0.43643578],
       [0.72045197, 0.62103443, 1.        , 0.46409548, 0.59914469],
       [0.75167814, 0.59160798, 0.46409548, 1.        , 0.14754222],
       [0.29534288, 0.43643578, 0.59914469, 0.14754222, 1.        ]])

In [40]:
for ingr in pancakes5.ingredients:
    print('*'*20)
    print(ingr)

********************
{'Сыворотка': '300 мл', 'Крупа манная': '2 ст. л.', 'Мука пшеничная': '1 стак.', 'Мука кукурузная': '1/2 стак.', 'Яйцо куриное': '1 шт', 'Масло растительное': '2 ст. л.', 'Дрожжи': '1 ч. л.', 'Паприка сладкая': '1 ч. л.', 'Брынза': '100 г', 'Пряности': '1 ч. л.', 'Сахар': '1 ч. л.', 'Порошок чесночный': '1 щепот.', 'Перец сладкий красный': '1/2 шт', 'Лук зеленый': '1 веточ.', 'Помидор': '1 шт', 'Кетчуп': '1 ст. л.', 'Йогурт': '150 г', 'Соль': 'по вкусу'}
********************
{'Помидор': '500 г', 'Мята': '1 пуч.', 'Мука пшеничная': '250 г', 'Лук репчатый': '1 шт', 'Сахар': '0,5 ч. л.', 'Перец душистый': '0,5 ч. л.', 'Соль': None, 'Масло оливковое': '3 ст. л.', 'Базилик': '0,5 ч. л.'}
********************
{'Филе куриное': '500 г', 'Сыр плавленый': '2 шт', 'Морковь': '1 шт', 'Кабачок': '300 г', 'Крупа манная': '1 ст. л.', 'Яйцо куриное': '1 шт', 'Специи': '2 ч. л.', 'Масло растительное': None}
********************
{'Хлопья злаковые': '1 стак.', 'Мука кукурузная': '1/3

In [41]:
df7 = pd.DataFrame(temp, columns=list(pancakes5.name), index=list(pancakes5.name))
df7

,Оладьи на балканский манер,Оладьи из помидоров с мятой,Нежные куриные оладьи,Оладьи мультизлаковые,"Оладьи из сельдерея, кабачков, феты и чечевицы"
Оладьи на балканский манер,1.00000,0.000000,0.000000,0.000000,0.115470
Оладьи из помидоров с мятой,0.00000,1.000000,0.046225,0.024441,0.030429
Нежные куриные оладьи,0.00000,0.046225,1.000000,0.203363,0.050637
Оладьи мультизлаковые,0.00000,0.024441,0.203363,1.000000,0.066935
"Оладьи из сельдерея, кабачков, феты и чечевицы",0.11547,0.030429,0.050637,0.066935,1.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [42]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    sim_df = sim_df.replace(1,0)
    recipe1 = sim_df.max().idxmax()
    recipe2 = sim_df.idxmax()[recipe1]
    return recipe1, recipe2

In [43]:
res = find_closest(df7)
df7.loc[res[0], res[1]]

0.9999999999999998

In [44]:
pancakes5.loc[pancakes5.name==res[0], 'description'].iloc[0]

'какой то хозяйственник проснулся в моём муже когда он принёс домой несколько килограммов помидоров теперь у нас дома начинаются томатные дни оладьи готовила полностью по рецепту от себя только щепотка базилика который как мне кажется помидорам всегда на пользу но я бы рецепт немного изменила выкладываю оригинальную версию со своими комментариями надеюсь вам понравится'

In [45]:
pancakes5.loc[pancakes5.name==res[1], 'description'].iloc[0]

'какой то хозяйственник проснулся в моём муже когда он принёс домой несколько килограммов помидоров теперь у нас дома начинаются томатные дни оладьи готовила полностью по рецепту от себя только щепотка базилика который как мне кажется помидорам всегда на пользу но я бы рецепт немного изменила выкладываю оригинальную версию со своими комментариями надеюсь вам понравится'